# Data Cleaning

In [1]:
import pandas as pd
import numpy as np
from pandarallel import pandarallel
import re

In [2]:
df = pd.read_parquet('arxiv.parquet', engine='pyarrow')
category_df = pd.read_parquet('categories.parquet', engine='pyarrow')

## Drop unneccessary columns

In [3]:
df_cleaned = df.drop(columns=['submitter', 'authors', 'update_date', 'comments',
                              'journal-ref', # many msising values, handled via journals added via API in feature engineering notebook
                              'report-no', 'license'] # mainly empty values
                              ).copy()

## Dropping records without doi

Records without DOIs were removed to filter out unpublished research and erroneous or incomplete submissions that have not been formally published. This approach was taken to ensure that only credible, peer-reviewed sources remain in the dataset.

In [4]:
print("Dropping rows with None in 'versions'...")
initial_rows = len(df_cleaned)

df_cleaned.dropna(subset=['doi'], inplace=True)
print(f"Dropped {initial_rows - len(df_cleaned)} rows.")

Dropping rows with None in 'versions'...
Dropped 1467846 rows.


## Dropping duplicate records (with the same doi)

In [5]:
print("Dropping duplicate records...")
initial_rows = len(df_cleaned)

df_cleaned = df_cleaned.drop_duplicates(subset='doi', keep='first') 
print(f"Dropped {initial_rows - len(df_cleaned)} rows.")

Dropping duplicate records...
Dropped 2244 rows.


## Filter to Papers created from 2015 onwards

In [6]:
def get_first_version_date(version_data):
    """
    Extracts the 'created' date from the FIRST dictionary in a list OR NumPy array,
    parses it using a specific format, and returns a timezone-aware datetime object (UTC).
    Returns NaT if input is invalid, empty, first item isn't dict,
    'created' key is missing, or date is unparseable with the given format.
    """
    # Imports needed within the function for parallel workers
    import numpy as np
    import pandas as pd

    is_list = isinstance(version_data, list)
    is_ndarray = isinstance(version_data, np.ndarray)

    # Check if it's a list/array and if it's NOT empty
    if not (is_list or is_ndarray) or len(version_data) == 0:
        return pd.NaT
    try:
        # --- Access the FIRST element ---
        first_version = version_data[0]
        # --------------------------------

        if not isinstance(first_version, dict):
            return pd.NaT # First item isn't a dictionary

        date_str = first_version.get('created')
        if date_str and isinstance(date_str, str):
            date_format = "%a, %d %b %Y %H:%M:%S %Z"
            # Parse using the specific format [[6]]
            dt = pd.to_datetime(date_str, format=date_format, errors='coerce', utc=True)
            return dt
        else:
            # 'created' key missing or value is not a string
            return pd.NaT
    except (IndexError, TypeError):
         # IndexError could happen if version_data[0] fails unexpectedly (though len check should prevent)
         # TypeError could happen if first_version is not subscriptable or .get fails
        return pd.NaT

# --- Pandarallel Initialization ---
print("Initializing Pandarallel...")
pandarallel.initialize(progress_bar=True)
print("Pandarallel initialized.")

# --- Apply using Pandarallel ---
print("Applying function using parallel_apply...")

# Apply the NEW function using parallel_apply
df_cleaned['first_date_parsed'] = df_cleaned['versions'].parallel_apply(get_first_version_date)

print("parallel_apply finished.")

# --- Ensure Consistent Datetime Type and Timezone ---
print("Converting to consistent datetime format...")
df_cleaned['first_date'] = pd.to_datetime(df_cleaned['first_date_parsed'], errors='coerce', utc=True)

# --- Final Filtering ---
print("Applying final filter...")
cutoff_date = pd.Timestamp('2015-01-01', tz='UTC')
# Use the 'first_date' column for comparison
mask = df_cleaned['first_date'] >= cutoff_date
df_cleaned = df_cleaned[mask].copy()

# Drop the temporary column
df_cleaned = df_cleaned.drop(columns=['first_date_parsed'])

# Drop the original 'versions' column
df_cleaned = df_cleaned.drop(columns=['versions'])



Initializing Pandarallel...
INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/
Pandarallel initialized.
Applying function using parallel_apply...


parallel_apply finished.
Converting to consistent datetime format...
Applying final filter...


## Handling Category Synonyms

In [7]:
# Synonyms dictionary
replacements = {
    "math.NA": "cs.NA",
    "eess.SY": "cs.SY",
    "math.IT": "cs.IT",
    "math.MP": "math-ph",
    "q-fin.EC": "econ.GN",
    "math.ST": "stat.TH"
}

# --- Count replacements ---

categories_to_count = df_cleaned['categories'].fillna('').astype(str)

total_replacements_count = 0

for old_cat, new_cat in replacements.items():
    # Escape the old category name for regex
    escaped_old_cat = re.escape(old_cat)
    # Create the regex pattern with word boundaries
    pattern = fr'\b{escaped_old_cat}\b'

    # Count occurrences of the pattern in the Series
    count_for_this_cat = categories_to_count.str.count(pattern).sum()

    # Add the count for this specific category to the total
    total_replacements_count += count_for_this_cat

for old_cat, new_cat in replacements.items():
    # Escape the old category name to handle special regex characters (like '.')
    escaped_old_cat = re.escape(old_cat)
    # Use regex with word boundaries (\b) to ensure only the exact category is replaced
    # fr'\b{escaped_old_cat}\b' creates a regex like r'\bmath\.NA\b'
    df_cleaned['categories'] = df_cleaned['categories'].str.replace(
        fr'\b{escaped_old_cat}\b',
        new_cat,
        regex=True
    )


In [8]:
def remove_duplicates_in_string(category_string):
    # Handle potential empty strings or NaN remnants if fillna wasn't perfect
    if not isinstance(category_string, str) or not category_string.strip():
        return "" # Return empty string for empty/invalid input

    # Split the string into a list of categories
    categories = category_string.split()

    # Use dict.fromkeys to get unique categories while preserving order
    # Filter out any potential empty strings that might result from multiple spaces
    unique_categories = list(dict.fromkeys(cat for cat in categories if cat))

    # Join the unique categories back into a space-separated string
    return ' '.join(unique_categories)

# Apply the function to the 'categories' column
df_cleaned['categories'] = df_cleaned['categories'].apply(remove_duplicates_in_string)

## Add Domain, Area and SubArea

In [9]:
category_lookup_df = category_df.set_index('Code')

# Convert relevant columns to a dictionary {Code: {col: value, ...}}
category_dict = category_lookup_df[['Domain', 'Area', 'SubArea']].to_dict('index')

# Define the function to get and aggregate info
def get_aggregated_info(category_string):
    if not isinstance(category_string, str) or not category_string.strip():
        return pd.Series({'Joined_Domain': '', 'Joined_Area': '', 'Joined_SubArea': ''})

    codes = category_string.split()
    domains = []
    areas = []
    subareas = []

    for code in codes:
        if code in category_dict:
            info = category_dict[code]
            domains.append(info['Domain'])
            areas.append(info['Area'])
            subareas.append(info['SubArea'])
        # else: handle codes not found in category_df if necessary (e.g., log warning)

    # Get unique values while preserving order and join with '; '
    joined_domain = '; '.join(list(dict.fromkeys(domains)))
    joined_area = '; '.join(list(dict.fromkeys(areas)))
    joined_subarea = '; '.join(list(dict.fromkeys(subareas)))

    return pd.Series({
        'Joined_Domain': joined_domain,
        'Joined_Area': joined_area,
        'Joined_SubArea': joined_subarea
    })

new_columns = df_cleaned['categories'].apply(get_aggregated_info)
df_cleaned = df_cleaned.join(new_columns) 

df_cleaned = df_cleaned.rename(columns={"Joined_Domain": "Domain", "Joined_Area": "Area", "Joined_SubArea": "SubArea"})

# Drop the original 'categories' column
df_cleaned = df_cleaned.drop(columns=['categories'])

# Drop Area (Only differs from SubArea for Physics, no broader Areas needed for Physics)
df_cleaned = df_cleaned.drop(columns=['Area'])
df_cleaned = df_cleaned.rename(columns={"SubArea": "Area"})

In [10]:
df_cleaned.to_parquet("arxiv_cleaned.parquet", engine='pyarrow', index=False)